<a href="https://colab.research.google.com/github/ItoMasaki/Notebook-List/blob/main/GraspWithYOLOv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install pixyz
import torch
import pixyz